# Backups

dbatools is **awesome** at performing backups :-)

The next block sets the variables for tje instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

## Investigate a directory using the SQL Service Account

We can investigate the file system from the viewpoint of the SQL Service Account using `Get-DbaFile` We are going to use that to show the files in the directory but this can be really useful when investigating errors with file system access, for example when backups fail or when reading from shares. This command will try ot access the path from teh SQL Instance using the SQL Account which means that you can troubleshoot the error and also that users do not need access to directories holding backups or client files for example  
If we look in the Directory that we created in the create container notebook using the command below

In [ ]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

You can see that all there is in the directory is a dummy file that we created earlier when we set up the directories  

## Check the last time that the databases were backed up

Knowing the last time a database was backed up is often useful information and with dbatools it can be retrieved easily. (NB - I use Warning action silently continue to hide the backups from multiple forks warnings that I have created when I created the container and havent fixed yet!)

In [ ]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

The databases were last backed up on the 21st December 2019. Thats not so clever!

## Docker requires permissions

You will get errors for the command below if the account that you have used to allow Docker to access your drives does not have permissions to the folder that we have created. [You can see the instructions here](https://docs.docker.com/docker-for-windows/)

You should add the docker account with full control over your dbatoolsdemo directory in your User Profile directory if you get errors below.

![DockerCompose](.\images\dbatoolsdemopermissions.png )

Its annoying but you can give the docker account modify permissions to your documents folder and it will inherited

![DockerCompose](.\images\documentsdirectorypermissions.png )



## Perform a backup of the entire instance

With one line of code we can quickly backup an entire instance.

In [ ]:
Backup-DbaDatabase  -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

All of the databases are backed up - Lets have a look in the directory

In [ ]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

If you want to have a look in the mounted volume on your laptop 

In [ ]:
Get-ChildItem "$FolderPath\SQL1"

## Backup to a directory for each database

Rob - I hear you cry, I have 2000 databases, I dont want them all backing up to the same directory - Can dbatools help me there ?

Sure, just add the `-CreateFolder` switch.

Lets demonstrate with SQL2

Theres nothing up my sleeves, lets look at the SQL2 folder


In [ ]:
Get-DbaFile -SqlInstance $SQL2 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL2

Only the dummy file! 

### Backup the databases

In [ ]:
Backup-DbaDatabase -SqlInstance $SQL2 -SqlCredential $SqlCredential -CreateFolder -Path /var/opt/mssql/backups/SQL2 -CompressBackup -WarningAction SilentlyContinue

## have a look in the directory

In [ ]:
Get-ChildItem "$FolderPath\SQL2" -Recurse

As you can see we have the databases backed up, each in their own directory :-)

## Double check all the instances

dbatoos works with multiple instances whereever there is a `-SqlInstance` parameter

Lets check the last time that these databases were backed up on both of the containers

In [ ]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

# We want to use Ola Hallengren for our backups

Thats ok, dbatools supports many of the excellent community tooling including Ola Hallengrens solution.

You can install Ola Hallengren like this (except in containers)

````
$installDbaMaintenanceSolutionSplat = @{
    SqlInstance = $sql1
    InstallJobs = $true
    ReplaceExisting = $true
    LogToTable = $true
    OutputFileDirectory = '/var/opt/mssql/backups/SQL1'
    BackupLocation = '/var/opt/mssql/backups/SQL1'
    CleanupTime = 72
    Solution = 'All'
    Database = 'master'
}
Install-DbaMaintenanceSolution @installDbaMaintenanceSolutionSplat -Verbose
````

Lets have a look at the Agent Jobs on the Instance

In [ ]:
Get-DbaAgentJob -SqlInstance $sql1 |Select Name

Those look like Ola Jobs to me :-)

Lets run the jobs to back up the system and user databases

In [ ]:
$Jobs = 'DatabaseBackup - SYSTEM_DATABASES - FULL' ,'DatabaseBackup - USER_DATABASES - FULL'
Start-DbaAgentJob -SqlInstance $sql1 -Job $Jobs

Check the status of the jobs

In [ ]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaAgentJob -SqlInstance $sql1 -Job $Jobs | Select SqlInstance, Name, CurrentRunStatus, LastRunOutCome

When the jobs have finished, lets have a look at the files.
We need to alter the file path to include the current container name from the results above

In [ ]:
Get-DbaFile -SqlInstance $sql1 -Path '/var/opt/mssql/data/ca2f813564a4/AdventureWorks2017/FULL'

# Restores

Backups are all very well and good but what about restores?

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason

In [ ]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

In [ ]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

All the databases have gone - How easy it is to restore them?

This easy :-)

One line of code

## First lets restore from the backups we took with dbatools in, with all files in the one directory

In [ ]:
Write-Output "Starting Restoring"
Restore-DbaDatabase  -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 
Write-Output "Finished"

In [ ]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

That was easy - WHat about if I have used Ola Hallengren?

AGAIN

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [ ]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

## Restore from Ola Halengren Backups 

- Use the path from the above files check with the new container name 

In [ ]:
Write-Output "Starting Restoring"
Restore-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/data/ca2f813564a4 -MaintenanceSolutionBackup
Write-Output "Finished"

In [ ]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

Thats all very well and good

## When were the databases restored?

and which backup was used?

In [ ]:
Get-DbaDbRestoreHistory -SqlInstance $SQL1 -Last

# Don't forget to clean up

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up